In [95]:
import csv 
import os 
from zipcodes import pullzipcodes

In [96]:
directory = os.environ['data_path']
files = sorted(os.listdir(directory))
df10 = directory + "/" + files[0]

In [97]:
zips = pullzipcodes()

####read RefUSA data and filter by zipcodes 

In [98]:
import pandas as pd

In [99]:
ref10 = pd.read_csv(df10, dtype = {'ZIP': object, 'CENSUS_TRACT_2010': object, 'CENSUS_BG_2010': object})

In [100]:
ref10.b = ref10[ref10['ZIP'].isin(zips)]

In [93]:
# ref10.b['CENSUS_TRACT_2010']

####read Borough Boundary Files from NYC Pluto 

In [8]:
import shapefile
from pyproj import Proj, transform 
from rtree import index 

In [102]:
#sf = shapefile.Reader("nybb_15b/nybb.shp")
sf = shapefile.Reader("nycb2010_15b/nycb2010.shp")
shapes = sf.shapes()

In [105]:
sf.record(0)


['1000',
 '5',
 'Staten Island',
 '000900',
 '50009001000',
 '2.50894811457e+003',
 '2.44589603884e+005']

In [121]:
sf.fields

[('DeletionFlag', 'C', 1, 0),
 ['CB2010', 'C', 4, 0],
 ['BoroCode', 'C', 1, 0],
 ['BoroName', 'C', 32, 0],
 ['CT2010', 'C', 6, 0],
 ['BCTCB2010', 'C', 11, 0],
 ['Shape_Leng', 'F', 19, 11],
 ['Shape_Area', 'F', 19, 11]]

In [12]:
# for i in range(5):
#     print sf.record(i)
def convert(li):
    f = 0.304800
    inProj = Proj(init='epsg:2263')
    outProj = Proj(init='epsg:4326')
    x0,y0 = li[0]*f, li[1]*f
    x1,y1 = li[2]*f, li[3]*f
    
    x2,y2 = transform(inProj,outProj,x0,y0)
    x3,y3 = transform(inProj,outProj,x1,y1)
    return (x2,y2,x3,y3)

In [13]:
convert(shapes[0].bbox)

(-74.08721257510494, 40.639801067600686, -74.0782078148904, 40.64479368862071)

In [14]:
for i, j in enumerate(sf.iterShapes()):
    if i < 20:
        print sf.record(i)[2], convert(j.bbox)
    else:
        break

Staten Island (-74.08721257510494, 40.639801067600686, -74.0782078148904, 40.64479368862071)
Staten Island (-74.12488226683051, 40.62304147709492, -74.11328569318664, 40.63267743930959)
Staten Island (-74.12063933549183, 40.634299203971096, -74.11721051820307, 40.636953037688926)
Staten Island (-74.13982129720739, 40.61278537625947, -74.13075780901775, 40.62038214872733)
Staten Island (-74.07788994742751, 40.596729078367204, -74.06106795735421, 40.60618022554547)
Staten Island (-74.08587722870362, 40.59215241304146, -74.07045220923452, 40.601947930425254)
Staten Island (-74.07043853539568, 40.58898696783525, -74.05965284458874, 40.598936199958445)
Staten Island (-74.13231400332512, 40.60815179961579, -74.12136466396817, 40.61307700429374)
Manhattan (-73.98383138039888, 40.71993342489936, -73.97823722668416, 40.72266941649322)
Manhattan (-73.98749853130087, 40.72267079443209, -73.97991324992839, 40.72817495451219)
Manhattan (-73.98730893624536, 40.73754477531876, -73.98086640741731, 40.

####Build rtree

In [106]:
idx = index.Index()

In [107]:
loc = 0 
for i in shapes:
    corners = shapes[loc].bbox 
    left, bottom, right, top = convert(corners)
    idx.insert(loc, (left, bottom, right, top))
    loc += 1

In [ ]:
def most_common(lst):
    return max(lst, key=lst.count)

In [76]:
def findBorough(row):
    a = row["LONGITUDE_2010"]
    b = row["LATITUDE_2010"]
    
    if pd.isnull(a) == True: 
        return 'NA' 
    
    else: 
        li = list(idx.intersection((a, b)))
        if len(li) != 0:
            li2 = [sf.record(ix)[2] for ix in li]
            bor = most_common(li2)
            return bor
        else: 
            return 'NA'
        

In [125]:
def findCT(row):
    a = row["LONGITUDE_2010"]
    b = row["LATITUDE_2010"]
    
    if pd.isnull(a) == True: 
        return 'NA' 
    else:
        li = list(idx.intersection((a, b)))
        
        if len(li) != 0:
            li2 = [sf.record(ix)[3] for ix in li]
            ct = most_common(li2)
            return ct

In [141]:
def findCB(row):
    a = row["LONGITUDE_2010"]
    b = row["LATITUDE_2010"]
    
    if pd.isnull(a) == True: 
        return 'NA' 
    else:
        li = list(idx.intersection((a, b)))
        
        if len(li) != 0:
            li2 = [sf.record(ix)[0][0] for ix in li]
            ct = most_common(li2)
            return ct

In [123]:
ref10.b['SHP_BOROUGH'] = ref10.b.apply(findBorough, axis=1)

/opt/rh/anaconda/root/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [126]:
ref10.b['SHP_CENSUS_TRACT'] = ref10.b.apply(findCT, axis=1)

/opt/rh/anaconda/root/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [142]:
ref10.b['SHP_CENSUS_BLOCK'] = ref10.b.apply(findCB, axis=1)

/opt/rh/anaconda/root/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [181]:
# # for i in range(20):
# #     li = list(idx.intersection((b[i], a[i])))
# #     li2 = [sf.record(ix)[2] for ix in li]
# #     print most_common(li2)

# for i in range(20):
#     li = list(idx.intersection((b[i], a[i])))
#     print [sf.record(ix)[0] for ix in li]

['1000', '8000']
['6001']
['2000', '1004', '1000']
['1001', '3000']
['3002', '3001', '3000']
['4000', '6001', '5001']
['3002', '3001']
['3001', '3000']
['3002', '3001', '6000']
['1000', '2000']
['3001', '3000']
['3001', '3000']
['2002', '1000']
['1001', '3000']
['4000', '6001', '5001']
['2000', '1000']
['3001', '3000']
['3001', '3000']
['4000', '6001', '5001']
['3002', '6000']


####read nys_acs data (contains New York, Richmond, Kings, Queens, and Bronx Counties) 

In [186]:
acs = pd.read_csv("nys_mi.csv", dtype = {'Geo_TRACT': object, 'Geo_BLKGRP': object, 'Geo_COUNTY': object})

In [190]:
counties = ['005', '047', '061', '081', '085']
#bronx 5 
#kings 47 
#New York 61 
#Queens 81
#Richmond 85 
acs.b = acs[acs['Geo_COUNTY'].isin(counties)][['Geo_NAME', 'Geo_COUNTY', 'Geo_TRACT', 'Geo_BLKGRP', 'SE_T057_001']]

In [199]:
acs.b.iloc[:,4].isnull().value_counts()

False    6193
True      300
dtype: int64

####filter RefUSA data

In [66]:
acs.b[(acs.b['Geo_TRACT'] == '026700') & (acs.b['Geo_BLKGRP'] == '2')]

,Geo_FIPS,Geo_GEOID,Geo_NAME,Geo_QName,Geo_STUSAB,Geo_SUMLEV,Geo_GEOCOMP,Geo_FILEID,Geo_LOGRECNO,Geo_US,...,Geo_SDUNI,Geo_UR,Geo_PCI,Geo_TAZ,Geo_UGA,Geo_BTTR,Geo_BTBG,Geo_PUMA5,Geo_PUMA1,SE_T057_001
4360,360470267002,15000US360470267002,"Block Group 2, Census Tract 267, Kings County,...","Block Group 2, Census Tract 267, Kings County,...",ny,150,0,ACSSF,12663,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47989
10676,360810267002,15000US360810267002,"Block Group 2, Census Tract 267, Queens County...","Block Group 2, Census Tract 267, Queens County...",ny,150,0,ACSSF,18979,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55729
